In [5]:
import warnings
import joblib
import pydotplus
import numpy as np
import pandas as pd
import seaborn as sns

In [6]:
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, cross_validate
from matplotlib import pyplot as plt
from sklearn.tree import DecisionTreeClassifier, export_graphviz, export_text
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate, validation_curve
from skompiler import skompile
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler, RobustScaler

In [7]:
df = pd.read_csv("Downloads/churn.csv")

In [8]:
df

,churn,accountlength,internationalplan,voicemailplan,numbervmailmessages,totaldayminutes,totaldaycalls,totaldaycharge,totaleveminutes,totalevecalls,totalevecharge,totalnightminutes,totalnightcalls,totalnightcharge,totalintlminutes,totalintlcalls,totalintlcharge,numbercustomerservicecalls
0,No,128,no,yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1
1,No,107,no,yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1
2,No,137,no,no,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0
3,No,84,yes,no,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2
4,No,75,yes,no,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,No,50,no,yes,40,235.7,127,40.07,223.0,126,18.96,297.5,116,13.39,9.9,5,2.67,2
4996,Yes,152,no,no,0,184.2,90,31.31,256.8,73,21.83,213.6,113,9.61,14.7,2,3.97,3
4997,No,61,no,no,0,140.6,89,23.90,172.8,128,14.69,212.4,97,9.56,13.6,4,3.67,1
4998,No,109,no,no,0,188.8,67,32.10,171.7,92,14.59,224.4,89,10.10,8.5,6,2.30,0


In [9]:
df.head()

,churn,accountlength,internationalplan,voicemailplan,numbervmailmessages,totaldayminutes,totaldaycalls,totaldaycharge,totaleveminutes,totalevecalls,totalevecharge,totalnightminutes,totalnightcalls,totalnightcharge,totalintlminutes,totalintlcalls,totalintlcharge,numbercustomerservicecalls
0,No,128,no,yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1
1,No,107,no,yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1
2,No,137,no,no,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0
3,No,84,yes,no,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2
4,No,75,yes,no,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3


In [10]:
def grab_col_names(dataframe, cat_th=10, car_th=20):
    """
    Veri setindeki kategorik, numerik ve kategorik fakat kordinal değişkenlerin isimlerini verir
    Parameters
    ----------
    dataframe: DataFrame
        değişkeni isimleri alınmak istenen DataFrame'dir
    cat_th: int, float
        numerik fakat kategorik olan değişkenler için eşik değeri
    car_th int, float
        numerik fakat kordinal olan değişenler için eşik değeri

    Returns
    -------
    cat_cols: list
        Kategorik değeşken listesi
    num_cols: list
        Numerik değişken listesi
    cat_but_car: list
        Kategorik görünümlü kordinal değişken listesi

    Notes
    -------
    cat_cols + num_cols + cat_but_car = toplam değişken sayısı
    num_but_cat cat_cols'un içinderdir

    """
    cat_cols = [col for col in dataframe.columns if str(dataframe[col].dtypes) in ["object", "category", "bool"]]

    num_but_cat = [col for col in dataframe.columns if dataframe[col].nunique() < 15 and dataframe[col].dtypes in ["int64", "float64", "int", "float"]]

    cat_but_car = [col for col in dataframe.columns if dataframe[col].nunique() > 20 and dataframe[col].dtypes in ["object", "category"]]

    cat_cols = cat_cols + num_but_cat

    cat_cols = [col for col in cat_cols if col not in cat_but_car]

    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes in ["int64", "float64", ]]

    num_cols = [col for col in num_cols if col not in num_but_cat]

    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f"cat_cols: {len(cat_cols)}")
    print(f"num_cols: {len(num_cols)}")
    print(f"cat_but_car: {len(cat_but_car)}")
    print(f"num_but_cat: {len(num_but_cat)}")

    return cat_cols, num_cols, cat_but_car

In [11]:
cat_cols, num_cols, cat_but_car = grab_col_names(df)

Observations: 5000
Variables: 18
cat_cols: 4
num_cols: 14
cat_but_car: 0
num_but_cat: 1


In [12]:
binary_cols = [col for col in df.columns if df[col].dtype not in ["int64", "float64"]
              and df[col].nunique() == 2]
binary_cols

['churn', 'internationalplan', 'voicemailplan']

In [17]:
df[binary_cols].head()

,churn,internationalplan,voicemailplan
0,0,0,1
1,0,0,1
2,0,0,0
3,0,1,0
4,0,1,0


In [14]:
def label_encoder (dataframe, binary_col):
    labelencoder = LabelEncoder()
    dataframe[binary_col] = labelencoder.fit_transform(dataframe[binary_col])
    return dataframe

In [15]:
for col in binary_cols:
    label_encoder(df, col)

In [16]:
mms = MinMaxScaler()
for col in num_cols:
    df[col] = mms.fit_transform(df[[col]])
df[num_cols].head()

,accountlength,numbervmailmessages,totaldayminutes,totaldaycalls,totaldaycharge,totaleveminutes,totalevecalls,totalevecharge,totalnightminutes,totalnightcalls,totalnightcharge,totalintlminutes,totalintlcalls,totalintlcharge
0,0.524793,0.480769,0.754196,0.666667,0.754183,0.542755,0.582353,0.542866,0.619494,0.520000,0.619584,0.500,0.15,0.500000
1,0.438017,0.500000,0.459744,0.745455,0.459672,0.537531,0.605882,0.537690,0.644051,0.588571,0.644344,0.685,0.15,0.685185
2,0.561983,0.000000,0.692461,0.690909,0.692436,0.333242,0.647059,0.333225,0.411646,0.594286,0.411930,0.610,0.25,0.609259
3,0.342975,0.000000,0.851778,0.430303,0.851740,0.170195,0.517647,0.170171,0.498481,0.508571,0.498593,0.330,0.35,0.329630
4,0.305785,0.000000,0.474253,0.684848,0.474230,0.407754,0.717647,0.407959,0.473165,0.691429,0.473270,0.505,0.15,0.505556


In [141]:
df.shape

(5000, 18)

In [142]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
churn,5000.0,0.141400,0.348469,0.0,0.000000,0.000000,0.000000,1.0
accountlength,5000.0,0.410160,0.164027,0.0,0.297521,0.409091,0.520661,1.0
internationalplan,5000.0,0.094600,0.292691,0.0,0.000000,0.000000,0.000000,1.0
voicemailplan,5000.0,0.264600,0.441164,0.0,0.000000,0.000000,1.000000,1.0
numbervmailmessages,5000.0,0.149138,0.260508,0.0,0.000000,0.000000,0.326923,1.0
totaldayminutes,5000.0,0.512913,0.153328,0.0,0.408819,0.512376,0.615078,1.0
totaldaycalls,5000.0,0.606239,0.120189,0.0,0.527273,0.606061,0.684848,1.0
totaldaycharge,5000.0,0.512879,0.153314,0.0,0.408802,0.512383,0.614960,1.0
totaleveminutes,5000.0,0.551654,0.138992,0.0,0.457451,0.552653,0.643662,1.0
totalevecalls,5000.0,0.589359,0.116626,0.0,0.511765,0.588235,0.670588,1.0


In [143]:
df = df.drop(["numbercustomerservicecalls"], axis = 1)

In [18]:
X = df.drop(["churn"], axis=1)

In [19]:
X

,accountlength,internationalplan,voicemailplan,numbervmailmessages,totaldayminutes,totaldaycalls,totaldaycharge,totaleveminutes,totalevecalls,totalevecharge,totalnightminutes,totalnightcalls,totalnightcharge,totalintlminutes,totalintlcalls,totalintlcharge,numbercustomerservicecalls
0,0.524793,0,1,0.480769,0.754196,0.666667,0.754183,0.542755,0.582353,0.542866,0.619494,0.520000,0.619584,0.500,0.15,0.500000,1
1,0.438017,0,1,0.500000,0.459744,0.745455,0.459672,0.537531,0.605882,0.537690,0.644051,0.588571,0.644344,0.685,0.15,0.685185,1
2,0.561983,0,0,0.000000,0.692461,0.690909,0.692436,0.333242,0.647059,0.333225,0.411646,0.594286,0.411930,0.610,0.25,0.609259,0
3,0.342975,1,0,0.000000,0.851778,0.430303,0.851740,0.170195,0.517647,0.170171,0.498481,0.508571,0.498593,0.330,0.35,0.329630,2
4,0.305785,1,0,0.000000,0.474253,0.684848,0.474230,0.407754,0.717647,0.407959,0.473165,0.691429,0.473270,0.505,0.15,0.505556,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0.202479,0,1,0.769231,0.670555,0.769697,0.670515,0.613143,0.741176,0.613394,0.753165,0.662857,0.753517,0.495,0.25,0.494444,2
4996,0.623967,0,0,0.000000,0.524040,0.545455,0.523929,0.706076,0.429412,0.706244,0.540759,0.645714,0.540799,0.735,0.10,0.735185,3
4997,0.247934,0,0,0.000000,0.400000,0.539394,0.399933,0.475117,0.752941,0.475251,0.537722,0.554286,0.537985,0.680,0.20,0.679630,1
4998,0.446281,0,0,0.000000,0.537127,0.406061,0.537149,0.472092,0.541176,0.472016,0.568101,0.508571,0.568374,0.425,0.30,0.425926,0


In [146]:
y = df["churn"]

In [147]:
y

0       0
1       0
2       0
3       0
4       0
       ..
4995    0
4996    1
4997    0
4998    0
4999    0
Name: churn, Length: 5000, dtype: int32

In [66]:
knn_model = KNeighborsClassifier().fit(X, y)

In [67]:
random_user = X.sample(1, random_state=45)

In [68]:
random_user

,accountlength,internationalplan,voicemailplan,numbervmailmessages,totaldayminutes,totaldaycalls,totaldaycharge,totaleveminutes,totalevecalls,totalevecharge,totalnightminutes,totalnightcalls,totalnightcharge,totalintlminutes,totalintlcalls,totalintlcharge
4988,0.326446,0,0,0.0,0.446657,0.612121,0.44662,0.5741,0.747059,0.574248,0.286835,0.622857,0.287001,0.81,0.1,0.809259


In [69]:
knn_model.predict(random_user)

array([0])

In [70]:
y_pred = knn_model.predict(X)

In [71]:
y_prob = knn_model.predict_proba(X)[:, 1]

In [72]:
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.99      0.95      4293
           1       0.92      0.46      0.61       707

    accuracy                           0.92      5000
   macro avg       0.92      0.73      0.78      5000
weighted avg       0.92      0.92      0.91      5000



In [73]:
roc_auc_score(y, y_prob)

0.9428939449800027

In [84]:
cv_results = cross_validate(knn_model, X, y, cv=5, scoring=["accuracy", "f1", "roc_auc"])

In [85]:
cv_results

{'fit_time': array([0.0246892 , 0.00400996, 0.00759006, 0.00858927, 0.00554776]),
 'score_time': array([0.21294165, 0.12824082, 0.16329336, 0.13185239, 0.08745098]),
 'test_accuracy': array([0.891, 0.891, 0.905, 0.891, 0.898]),
 'test_f1': array([0.47342995, 0.44102564, 0.53658537, 0.45226131, 0.50961538]),
 'test_roc_auc': array([0.73553282, 0.76755918, 0.79888374, 0.76220083, 0.77906776])}

In [86]:
cv_results["test_accuracy"].mean()

0.8952

In [87]:
cv_results["test_f1"].mean()

0.48258352994363374

In [88]:
cv_results["test_roc_auc"].mean()

0.7686488666857643

In [89]:
knn_model = KNeighborsClassifier()

In [90]:
knn_model

KNeighborsClassifier()

In [91]:
knn_model.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

In [95]:
knn_params = {"n_neighbors": range(2, 500)}

In [96]:
knn_gs_best = GridSearchCV(knn_model,
                           knn_params,
                           cv=5,
                           n_jobs=-1,
                           verbose=1).fit(X, y)

Fitting 5 folds for each of 498 candidates, totalling 2490 fits


In [97]:
knn_gs_best.best_params_

{'n_neighbors': 9}

In [98]:
knn_final = knn_model.set_params(**knn_gs_best.best_params_).fit(X, y)

In [99]:
cv_results = cross_validate(knn_final,
                            X,
                            y,
                            cv=5,
                            scoring=["accuracy", "f1", "roc_auc"])

In [100]:
cv_results

{'fit_time': array([0.04016948, 0.00457597, 0.00400376, 0.00300765, 0.0030427 ]),
 'score_time': array([0.1428225 , 0.07754135, 0.07103777, 0.06983566, 0.06904888]),
 'test_accuracy': array([0.892, 0.893, 0.903, 0.895, 0.899]),
 'test_f1': array([0.44329897, 0.44559585, 0.4921466 , 0.46153846, 0.49751244]),
 'test_roc_auc': array([0.75060478, 0.77920888, 0.8140878 , 0.80136413, 0.80961292])}

In [101]:
cv_results["test_accuracy"].mean()

0.8964000000000001

In [102]:
cv_results["test_f1"].mean()

0.468018464040498

In [103]:
cv_results["test_roc_auc"].mean()

0.7909757006817236

In [104]:
random_user = X.sample(1)

In [105]:
knn_final.predict(random_user)

array([0])

## cart model

In [148]:
cart_model = DecisionTreeClassifier(random_state=1).fit(X, y)

In [149]:
y_pred = cart_model.predict(X)

In [150]:
y_prob = cart_model.predict_proba(X)[:, 1]

In [151]:
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4293
           1       1.00      1.00      1.00       707

    accuracy                           1.00      5000
   macro avg       1.00      1.00      1.00      5000
weighted avg       1.00      1.00      1.00      5000



In [152]:
roc_auc_score(y, y_prob)

1.0

# holdout yöntemi

In [153]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=17)

In [154]:
cart_model = DecisionTreeClassifier(random_state=17).fit(X_train, y_train)

In [155]:
#train hatası
y_pred = cart_model.predict(X_train)

In [156]:
y_prob = cart_model.predict_proba(X_train)[:, 1]

In [157]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3026
           1       1.00      1.00      1.00       474

    accuracy                           1.00      3500
   macro avg       1.00      1.00      1.00      3500
weighted avg       1.00      1.00      1.00      3500



In [158]:
roc_auc_score(y_train, y_prob)

1.0

In [159]:
### Test hatası
y_pred = cart_model.predict(X_test)

In [160]:
y_prob = cart_model.predict_proba(X_test)[:, 1]

In [161]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.91      0.91      1267
           1       0.52      0.55      0.53       233

    accuracy                           0.85      1500
   macro avg       0.72      0.73      0.72      1500
weighted avg       0.85      0.85      0.85      1500



In [162]:
roc_auc_score(y_test, y_prob)

0.7273221526298139

# cv ile başarı değerlendirme

In [163]:
cart_model = DecisionTreeClassifier(random_state=17).fit(X, y)

In [164]:
cv_results = cross_validate(cart_model,
                            X,y,
                            cv=5,
                            scoring=["accuracy", "f1", "roc_auc"])

In [165]:
cv_results["test_accuracy"].mean()

0.8762000000000001

In [166]:
cv_results["test_f1"].mean()

0.5741705475850575

In [167]:
cv_results["test_roc_auc"].mean()

0.7560095227747016

# Hiperparametre Optimizasyonu

In [168]:
cart_model.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'random_state': 17,
 'splitter': 'best'}

In [169]:
cart_params = {"max_depth": range(1, 11),
               "min_samples_split": range(2, 20)}

In [170]:
cart_best_grid = GridSearchCV(cart_model,
                              cart_params,
                              n_jobs=-1,
                              verbose=1).fit(X, y)

Fitting 5 folds for each of 180 candidates, totalling 900 fits


In [171]:
cart_best_grid.best_params_

{'max_depth': 8, 'min_samples_split': 7}

In [172]:
cart_best_grid.best_score_

0.9246000000000001

In [173]:
random = X.sample(1, random_state=45)

In [174]:
cart_best_grid.predict(random)

array([0])

# final model

In [175]:
cart_final = DecisionTreeClassifier(**cart_best_grid.best_params_, random_state=17).fit(X, y)

In [176]:
cart_final.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 8,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 7,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'random_state': 17,
 'splitter': 'best'}

In [177]:
cart_final = cart_model.set_params(**cart_best_grid.best_params_).fit(X, y)

In [178]:
cart_final.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 8,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 7,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'random_state': 17,
 'splitter': 'best'}

In [179]:
cv_results = cross_validate(cart_model,
                            X,y,
                            cv=5,
                            scoring=["accuracy", "f1", "roc_auc"])

In [180]:
cv_results["test_accuracy"].mean()

0.9246000000000001

In [181]:
cv_results["test_f1"].mean()

0.6705137567746264

In [182]:
cv_results["test_roc_auc"].mean()

0.7863771520084064